In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
open_spaces = pd.read_csv('data/locations/open_spaces.csv')
open_spaces = open_spaces.loc[~open_spaces['Type'].isin(['Cemeteries & Burying Grounds', 'Urban Wilds', 'Open Land', 'Other Open Land'])]
canopy = pd.read_csv('data/locations/open_spaces_with_canopy.csv')
streetlights = pd.read_csv('data/locations/open_spaces_with_streetlights.csv')
crimes = pd.read_csv('data/locations/open_spaces_with_crimes.csv')
nearest_bluebike_stations = pd.read_csv('nearest_bluebike_stations_k_20_d_0.5.csv')
nearest_restaurants = pd.read_csv('nearest_restaurants_k_100_d_0.5.csv')
nearest_food_trucks = pd.read_csv('nearest_food_trucks_k_10_d_0.5.csv')
nearest_farmers_markets = pd.read_csv('nearest_farmers_markets_k_7_d_0.5.csv')
nearest_murals = pd.read_csv('nearest_murals_k_10_d_0.5.csv')
roundtrips = pd.read_csv('roundtrips.csv')
park_assets = pd.read_csv('data/locations/park_assets.csv')

In [3]:
selected = [
    'Boston Common',
    'Chinatown Park',
    'City Hall Plaza',
    'Dewey Square Parks',
    'Fallon Field',
    'Faneuil Square',
    'Olmsted Park',
    'Ronan Park',
    'Ross Playground',
    'Carson Beach'
]

In [4]:
open_spaces = open_spaces.loc[open_spaces['Name'].isin(selected)]
open_spaces

,SPACEID,Name,Type,Acres,Geometry,Avg. Lat,Avg. Long,Lat Range,Long Range
24,25,Boston Common,Parks,45.740950,MULTIPOLYGON (((-71.06325725499994 42.35760757...,42.355043,-71.065667,0.005130,0.007050
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,MULTIPOLYGON (((-71.05865943099997 42.36071787...,42.360502,-71.058934,0.002333,0.002581
121,122,Fallon Field,Athletic Fields,7.512146,MULTIPOLYGON (((-71.13383823799995 42.28732795...,42.287572,-71.135249,0.001399,0.003204
159,160,Ronan Park,Parks,11.241366,MULTIPOLYGON (((-71.06341298199999 42.30565509...,42.304354,-71.062631,0.002601,0.003665
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,MULTIPOLYGON (((-71.05648443799998 42.36033046...,42.360045,-71.056464,0.000571,0.001081
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,MULTIPOLYGON (((-71.05865915699997 42.35217557...,42.351329,-71.059101,0.002085,0.001535
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,MULTIPOLYGON (((-71.05163775199998 42.35475411...,42.354103,-71.053536,0.001998,0.003796
423,424,Olmsted Park,Parks,42.825230,MULTIPOLYGON (((-71.11463093299994 42.32455331...,42.325980,-71.115734,0.009777,0.007450
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,MULTIPOLYGON (((-71.03837250899994 42.32975120...,42.326421,-71.041152,0.007558,0.015440
483,484,Ross Playground,Playgrounds,13.124954,MULTIPOLYGON (((-71.11254901299998 42.26544443...,42.264119,-71.112935,0.002650,0.003789


In [5]:
park_assets['Feature List'] = park_assets['Feature(s)'].apply(str.split, sep=', ')
playgrounds = set(park_assets.loc[park_assets['Feature List'].apply(lambda x: 'Playground' in x), 'Park'])
athletic_fields = set(park_assets.loc[park_assets['Feature List'].apply(lambda x: 'Athletic Field' in x), 'Park'])

In [6]:
descriptions = [
    "America's oldest public park dating back to 1634, this beautiful urban greenspace hosts events and recreation.",
    "Expansive open space hosting concerts, festivals, and events, surrounded by iconic brutalist architecture in downtown Boston.",
    "Spacious public playing field offering recreational opportunities in Roslindale neighborhood.",
    "Ronan Park is a park in Dorchester’s Fields Corner neighborhood. The park is located on a hill and has many sports amenities, including a basketball area with many hoops, a big league baseball field, and walking paths. The park has a fenced-in dog area as well.",
    "Vibrant outdoor marketplace featuring street performers, colonial history, diverse eateries, and lively atmosphere in downtown Boston.",
    "Small urban oasis featuring Chinese-style gateway, plantings, and seating areas amid Chinatown's bustle.",
    "Vibrant public plaza at the end of the Rose Kennedy Greenway right outside of South Station known for its food trucks.",
    "The Emerald Necklace's largest park, designed by landscape architect Frederick Law Olmsted, featuring meadows, waterways, and woodlands.",
    "test",
    "Family-friendly neighborhood park with playground equipment, lawns, and sports courts in Hyde Park."
]

In [7]:
data = open_spaces.loc[:, 'SPACEID':'Acres']

data['Description'] = descriptions

data['Acres (Scaled)'] = np.clip(np.log10(data['Acres'] + 1) / np.log10(51), 0, 1)

data['Canopy Cover Proportion'] = canopy['Canopy Cover Proportion']

data['Streetlight Proportion'] = streetlights['Streetlight Cover Proportion']
data['Nearby Crimes Per Acre Per Year'] = crimes['Nearby Crimes Per Acre Per Year']
data['Nearby Vehicle Accidents Per Acre Per Year'] = crimes['Nearby Vehicle Accidents Per Acre Per Year']

data['Nearby Crimes Per Acre Per Year (Scaled)'] = np.clip(np.sqrt(1 - np.clip(data['Nearby Crimes Per Acre Per Year'], 0, 8) / 8), 0, 1)
data['Nearby Vehicle Accidents Per Acre Per Year (Scaled)'] = np.clip(np.sqrt(1 - np.clip(data['Nearby Vehicle Accidents Per Acre Per Year'], 0, 2) / 2), 0, 1)

data['Number of Nearby Bluebike Stations'] = nearest_bluebike_stations['num_objs']
data[['Daily Roundtrips', 'Weighted Roundtrip Ratio']] = roundtrips[['Daily Roundtrips', 'Weighted Roundtrip Ratio']]

data['Number of Nearby Bluebike Stations (Scaled)'] = np.clip(np.log10(data['Number of Nearby Bluebike Stations'] + 1)/np.log10(11), 0, 1)
data['Daily Roundtrips (Scaled)'] = np.clip(np.log10(data['Daily Roundtrips'] + 1) / np.log10(25), 0, 1)
data['Weighted Roundtrip Ratio (Scaled)'] = np.clip(np.log10(data['Weighted Roundtrip Ratio'] + 1) / np.log10(1.25), 0, 1)

data['Number of Nearby Restaurants'] = (nearest_restaurants['num_objs']).astype(int)
data['Near Food Trucks'] = nearest_food_trucks['num_objs'] > 0
data['Near Farmers Markets'] = nearest_farmers_markets['num_objs'] > 0

data['Number of Nearby Restaurants (Scaled)'] = np.clip(np.log10(data['Number of Nearby Restaurants'] + 1) / np.log10(51), 0, 1)

data['Has Athletic Field'] = data['Name'].isin(athletic_fields)
data['Has Playground'] = data['Name'].isin(playgrounds)

data['Near Murals'] = nearest_murals['num_objs'] > 0

data

,SPACEID,Name,Type,Acres,Description,Acres (Scaled),Canopy Cover Proportion,Streetlight Proportion,Nearby Crimes Per Acre Per Year,Nearby Vehicle Accidents Per Acre Per Year,...,Number of Nearby Bluebike Stations (Scaled),Daily Roundtrips (Scaled),Weighted Roundtrip Ratio (Scaled),Number of Nearby Restaurants,Near Food Trucks,Near Farmers Markets,Number of Nearby Restaurants (Scaled),Has Athletic Field,Has Playground,Near Murals
24,25,Boston Common,Parks,45.740950,America's oldest public park dating back to 16...,0.977821,0.357298,0.965003,4.282123,0.762686,...,1.000000,1.000000,0.192431,100,False,False,1.000000,True,True,True
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,"Expansive open space hosting concerts, festiva...",0.495420,0.129559,0.870738,5.963339,0.942258,...,1.000000,0.809160,0.128825,100,True,False,1.000000,False,True,True
121,122,Fallon Field,Athletic Fields,7.512146,Spacious public playing field offering recreat...,0.544656,0.145290,0.758661,0.294724,0.213193,...,0.578130,0.192315,0.549124,24,False,True,0.818672,True,True,True
159,160,Ronan Park,Parks,11.241366,Ronan Park is a park in Dorchester’s Fields Co...,0.637063,0.221674,0.860189,1.868008,0.934099,...,0.458157,0.189479,0.439005,40,False,False,0.944491,True,True,True
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,Vibrant outdoor marketplace featuring street p...,0.151856,0.013949,1.000000,5.113267,0.978841,...,0.916314,0.800427,0.155235,100,True,False,1.000000,False,False,True
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,Small urban oasis featuring Chinese-style gate...,0.155357,0.512517,1.000000,5.120670,0.769895,...,0.811508,0.727116,0.125777,100,False,False,1.000000,False,True,True
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,Vibrant public plaza at the end of the Rose Ke...,0.328628,0.422465,0.959966,3.135443,0.608770,...,0.867194,0.701672,0.102007,100,False,False,1.000000,False,False,True
423,424,Olmsted Park,Parks,42.825230,"The Emerald Necklace's largest park, designed ...",0.961439,0.612260,0.297061,0.307462,0.170354,...,0.458157,0.278496,0.174304,15,False,False,0.705166,True,False,True
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,test,0.825165,0.112254,0.311006,0.419466,0.132473,...,0.578130,0.559778,0.596150,8,False,False,0.558831,False,False,True
483,484,Ross Playground,Playgrounds,13.124954,Family-friendly neighborhood park with playgro...,0.673464,0.177787,0.349196,0.350146,0.223502,...,0.289065,0.031155,1.000000,3,False,False,0.352583,True,True,True


In [8]:
data[['Nearby Crimes Per Acre Per Year', 'Nearby Crimes Per Acre Per Year (Scaled)', 'Nearby Vehicle Accidents Per Acre Per Year', 'Nearby Vehicle Accidents Per Acre Per Year (Scaled)']]

,Nearby Crimes Per Acre Per Year,Nearby Crimes Per Acre Per Year (Scaled),Nearby Vehicle Accidents Per Acre Per Year,Nearby Vehicle Accidents Per Acre Per Year (Scaled)
24,4.282123,0.681714,0.762686,0.786548
40,5.963339,0.504562,0.942258,0.727235
121,0.294724,0.981407,0.213193,0.945200
159,1.868008,0.875499,0.934099,0.730035
279,5.113267,0.600701,0.978841,0.714548
307,5.120670,0.599930,0.769895,0.784253
310,3.135443,0.779788,0.608770,0.834035
423,0.307462,0.980595,0.170354,0.956464
477,0.419466,0.973430,0.132473,0.966314
483,0.350146,0.977871,0.223502,0.942470


In [9]:
data['Environment Score'] = round(
    30 * data['Acres (Scaled)'] +
    70 * data['Canopy Cover Proportion']
).astype(int)

data['Safety Score'] = round(
    30 * data['Streetlight Proportion'] + 
    35 * data['Nearby Crimes Per Acre Per Year (Scaled)'] +
    35 * data['Nearby Vehicle Accidents Per Acre Per Year (Scaled)']
    ).astype(int)

data['Activity Score'] = round(
    10 * data['Number of Nearby Bluebike Stations (Scaled)'] +
    80 * data['Daily Roundtrips (Scaled)'] +
    10 * data['Weighted Roundtrip Ratio (Scaled)']
).astype(int)

data['Food Score'] = round(
    60 * data['Number of Nearby Restaurants (Scaled)'] +
    20 * data['Near Food Trucks'] +
    20 * data['Near Farmers Markets']
).astype(int)

data['Play Score'] = round(
    50 * data['Has Athletic Field'] +
    50 * data['Has Playground']
).astype(int)

data['Art Score'] = round(
    100 * data['Near Murals']
).astype(int)

data['Image'] = data['Name'].apply(lambda x: f"assets/{''.join(x.split()).replace('/', '-')}.jpeg")

data

,SPACEID,Name,Type,Acres,Description,Acres (Scaled),Canopy Cover Proportion,Streetlight Proportion,Nearby Crimes Per Acre Per Year,Nearby Vehicle Accidents Per Acre Per Year,...,Has Athletic Field,Has Playground,Near Murals,Environment Score,Safety Score,Activity Score,Food Score,Play Score,Art Score,Image
24,25,Boston Common,Parks,45.740950,America's oldest public park dating back to 16...,0.977821,0.357298,0.965003,4.282123,0.762686,...,True,True,True,54,80,92,60,100,100,assets/BostonCommon.jpeg
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,"Expansive open space hosting concerts, festiva...",0.495420,0.129559,0.870738,5.963339,0.942258,...,False,True,True,24,69,76,80,50,100,assets/CityHallPlaza.jpeg
121,122,Fallon Field,Athletic Fields,7.512146,Spacious public playing field offering recreat...,0.544656,0.145290,0.758661,0.294724,0.213193,...,True,True,True,27,90,27,69,100,100,assets/FallonField.jpeg
159,160,Ronan Park,Parks,11.241366,Ronan Park is a park in Dorchester’s Fields Co...,0.637063,0.221674,0.860189,1.868008,0.934099,...,True,True,True,35,82,24,57,100,100,assets/RonanPark.jpeg
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,Vibrant outdoor marketplace featuring street p...,0.151856,0.013949,1.000000,5.113267,0.978841,...,False,False,True,6,76,75,80,0,100,assets/FaneuilSquare.jpeg
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,Small urban oasis featuring Chinese-style gate...,0.155357,0.512517,1.000000,5.120670,0.769895,...,False,True,True,41,78,68,60,50,100,assets/ChinatownPark.jpeg
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,Vibrant public plaza at the end of the Rose Ke...,0.328628,0.422465,0.959966,3.135443,0.608770,...,False,False,True,39,85,66,60,0,100,assets/DeweySquareParks.jpeg
423,424,Olmsted Park,Parks,42.825230,"The Emerald Necklace's largest park, designed ...",0.961439,0.612260,0.297061,0.307462,0.170354,...,True,False,True,72,77,29,42,50,100,assets/OlmstedPark.jpeg
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,test,0.825165,0.112254,0.311006,0.419466,0.132473,...,False,False,True,33,77,57,34,0,100,assets/CarsonBeach.jpeg
483,484,Ross Playground,Playgrounds,13.124954,Family-friendly neighborhood park with playgro...,0.673464,0.177787,0.349196,0.350146,0.223502,...,True,True,True,33,78,15,21,100,100,assets/RossPlayground.jpeg


In [10]:
data.set_index('Name').to_json('final_data_table.json', orient='index')